In [1]:
import itertools as it
import os
import json
import requests
import bs4
import geocoder
import folium
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
pd.set_option('display.max_columns', None)

# Neighborhood dataframe creation

I decided to cluster neighborhoods instead of boroughs or postal codes, so I went ahead and split postal codes with multiple neighborhoods into multiple rows.

In [2]:
soup = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").content
soup = BeautifulSoup(soup)

In [3]:
data = pd.DataFrame(columns=['Postal Code', 'Borough', 'Neighborhood'])
table = soup.find('table')
for cell in table.find_all('td'):
    row = list(cell.stripped_strings)
    if "Not assigned" in row:
        continue
    neighborhoods = " ".join(row[2:]).strip("() ")
    neighborhoods = " ".join([t.strip() for t in neighborhoods.split(")")])
    neighborhoods = neighborhoods.replace("(", "/")
    neighborhoods = ", ".join([t.strip() for t in neighborhoods.split(",")])
    neighborhoods = ", ".join([t.strip() for t in neighborhoods.split("/")])
    for nbh in neighborhoods.split(','):
        data = data.append(dict(zip(data.columns, row[:2] + [nbh.strip()])), ignore_index=True)
data.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M5A,Downtown Toronto,Harbourfront
4,M6A,North York,Lawrence Manor


# Adding Latitude/Longitude

Latitude, longitude, and radius will be added to the data frame created above. The radius used for FourSquare exploration will be that of a circle inscribed in the bounding box of the postal code area.

In [4]:
def save_locations(data):
    latitude = []
    longitude = []
    radius = []
    for nbh, pc in zip(data['Neighborhood'], data['Postal Code']):
        print("Geocoding", nbh)
        for i in range(10):
            response = None
            response = geocoder.arcgis("{} Toronto Canada {}".format(nbh, pc))
            if response is not None:
                break
        else:
            if response is None:
                print("Too many lookup attempts. The geocoding server may be down...")
        latitude.append(response.lat)
        longitude.append(response.lng)
        NE = response.bbox['northeast']
        SW = response.bbox['southwest']
        NW = [NE[0], SW[1]]
        radius.append(min(geocoder.distance([NE, NW], units='meters'), geocoder.distance([NW, SW], units='meters')) / 2)

    latitude = pd.Series(latitude, name='Latitude')
    longitude = pd.Series(longitude, name='Longitude')
    radius = pd.Series(radius, name="Radius")
    data[["Latitude"]] = latitude
    data[["Longitude"]] = longitude
    data[["Radius"]] = radius
    data.to_csv("toronto_neighborhoods.csv")
    return data

Load from previously saved data frame if we can, otherwise run the function above and save the updated data frame.

In [5]:
if os.access("toronto_neighborhoods.csv", os.R_OK):
    data = pd.read_csv("toronto_neighborhoods.csv", index_col=0)
else:
    data = save_locations(data)

In [6]:
data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Radius
0,M3A,North York,Parkwoods,43.752450,-79.329910,2568.862665
1,M4A,North York,Victoria Village,43.730570,-79.313060,1445.852272
2,M5A,Downtown Toronto,Regent Park,43.659743,-79.361561,80.442896
3,M5A,Downtown Toronto,Harbourfront,43.639230,-79.383070,402.325087
4,M6A,North York,Lawrence Manor,43.714925,-79.449739,80.368924


# Neighborhood Exploration

Load API key and secret and create URL template for calls to the FourSquare exploration endpoint.

In [7]:
if os.access('foursquare_api_keys.txt', os.R_OK):
    with open("foursquare_api_keys.txt") as f:
        foursquare_id = f.readline().strip()
        foursquare_secret = f.readline().strip()
else:
    foursquare_id = ""
    foursquare_secret = ""
foursquare_template = "https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v=20210526".format("{}", 
                                                                                                                foursquare_id, 
                                                                                                                foursquare_secret)
foursquare_explore = foursquare_template.format("explore") + "&limit={}".format(100) + "&ll={},{}&radius={}"

Get top 100 venues for each neighborhood using the Foursquare API and parse response to build a data frame.

In [8]:
def get_venues(data):
    all_venues = pd.DataFrame()
    for i in range(data.shape[0]):
        lat, lng, rad = data[['Latitude', 'Longitude', 'Radius']].iloc[i]
        url = foursquare_explore.format(lat, lng, rad)
        response = requests.api.get(url)
        response.raise_for_status()
        venues = response.json()['response']['groups'][0]["items"]
        venues = pd.json_normalize(venues)
        venues['Neighborhood'] = data.loc[i, 'Neighborhood']
        try:
            venues['reasons.items'] = venues['reasons.items'].map(lambda x: [i['reasonName'] for i in x][0])
        except (TypeError, KeyError):
            pass
        try:
            venues['venue.categories.ids'] = venues['venue.categories'].map(lambda x: [i['id'] for i in x][0])
            venues['venue.categories.names'] = venues['venue.categories'].map(lambda x: [i['name'] for i in x][0])
        except KeyError:
            pass
        all_venues = all_venues.append(venues, ignore_index=True)
    all_venues.to_csv('toronto_neighborhood_venues.csv')
    return all_venues

In [9]:
if os.access('toronto_neighborhood_venues.csv', os.R_OK):
    venues = pd.read_csv('toronto_neighborhood_venues.csv', index_col=0)
else:
    venues = get_venues(data)

keep_cols = ['Neighborhood', 'venue.id', 'venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng',  
             'venue.location.distance', 'venue.categories.names', 'venue.categories.ids']
new_col_names = ['Neighborhood', 'ID', 'Name', 'Address', 'Latitude', 'Longitude', 'Distance', 'Category', 'Category ID']
venues = venues[keep_cols]
venues.columns = new_col_names
venues.head()

,Neighborhood,ID,Name,Address,Latitude,Longitude,Distance,Category,Category ID
0,Parkwoods,4b8991cbf964a520814232e3,Allwyn's Bakery,81 Underhill drive,43.759840,-79.324719,922.0,Caribbean Restaurant,4bf58dd8d48988d144941735
1,Parkwoods,57e286f2498e43d84d92d34a,Tim Hortons,215 Brookbanks,43.760668,-79.326368,958.0,Café,4bf58dd8d48988d16d941735
2,Parkwoods,4bd4846a6798ef3bd0c5618d,Donalda Golf & Country Club,12 Bushbury Dr,43.752816,-79.342741,1032.0,Golf Course,4bf58dd8d48988d1e6941735
3,Parkwoods,4e8d9dcdd5fbbbb6b3003c7b,Brookbanks Park,Toronto,43.751976,-79.332140,186.0,Park,4bf58dd8d48988d163941735
4,Parkwoods,4b8ec91af964a520053733e3,Graydon Hall Manor,185 Graydon Hall Drive,43.763923,-79.342961,1652.0,Event Space,4bf58dd8d48988d171941735


If a venue is listed in multiple neighborhoods, keep the one with the minimum distance. This ensures each venue is only associated with one neighborhood and offsets any neighborhood overlap when basing the search radius on neighborhood bounding boxes.

In [10]:
venue_id_groups = venues.groupby('ID')
min_dist_idx = venue_id_groups['Distance'].idxmin()
venue_id_groups['Distance'].min()
venues = venues.iloc[min_dist_idx]
venues.set_index('ID', inplace=True)
venues.head()

,Neighborhood,Name,Address,Latitude,Longitude,Distance,Category,Category ID
ID,,,,,,,,
4a8d5b48f964a520840f20e3,Kensington Market,The Moonbean Cafe,30 St Andrew St,43.654147,-79.400182,117.0,Café,4bf58dd8d48988d16d941735
4a9d0a53f964a520d23720e3,Business reply mail Processing Centre 969 Eastern,Hilton,145 Richmond St W,43.649946,-79.385479,139.0,Hotel,4bf58dd8d48988d1fa931735
4aada317f964a5201a6120e3,Yorkville,Isabel Bader Theatre,93 Charles St W,43.667329,-79.392604,382.0,College Theater,4bf58dd8d48988d1ac941735
4ab17387f964a520866920e3,St. James Town,Churchmouse & Firkin,485 Church St.,43.664632,-79.380406,729.0,Pub,4bf58dd8d48988d11b941735
4ab81980f964a5202f7c20e3,The Beaches West,The Salty Dog,1982 Queen St E,43.669944,-79.300894,193.0,Restaurant,4bf58dd8d48988d1c4941735


# Convert venue category into indicator variables

In [11]:
categories = venues['Category']
categories = pd.get_dummies(categories, columns=['Category'], prefix='', prefix_sep='')
categories.drop(columns='Neighborhood', inplace=True)
categories = pd.concat([venues['Neighborhood'], categories], axis=1)
categories.head()

,Neighborhood,ATM,Accessories Store,African Restaurant,Airport,Airport Lounge,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Home Service,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Leather Goods Store,Library,Light Rail Station,Lighting Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Restaurant,River,Road,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,

# Group venues by neighborhood and get category frequencies

In [12]:
category_counts = categories.groupby('Neighborhood').mean()

# Determine top 5 most popular types of places for each neighborhood

In [13]:
def get_top5(category_counts):
    top5 = pd.DataFrame()
    for nbh_id in category_counts.index:
        row = category_counts.loc[nbh_id].nlargest(5).map(lambda x: x if  x > 0 else np.nan)
        row = pd.Series(row.index, name=nbh_id)
        top5 = pd.concat([top5, row], axis=1)
    top5 = top5.T
    top5.index.name = 'Neighborhood'
    top5.columns = ["{}Most Popular".format(n) for n in ['', '2nd ', '3rd ', '4th ', '5th ']]
    return top5
top5 = get_top5(category_counts)
top5.head()

,Most Popular,2nd Most Popular,3rd Most Popular,4th Most Popular,5th Most Popular
Neighborhood,,,,,
Adelaide,Salad Place,Thai Restaurant,ATM,Accessories Store,African Restaurant
Agincourt North,Chinese Restaurant,Coffee Shop,Bakery,Sandwich Place,Bubble Tea Shop
Alderwood,Coffee Shop,Bank,Bar,Café,Discount Store
Bathurst Manor,Coffee Shop,Deli / Bodega,Middle Eastern Restaurant,Restaurant,Sandwich Place
Bayview Village,Chinese Restaurant,Bank,Café,Clothing Store,Coffee Shop


# Cluster neighborhoods

Cluster neighborhoods based on venue category frequencies

In [14]:
def cluster_label(category_counts):
    km = KMeans(n_clusters=5, n_init=1000)
    km.fit(category_counts)
    labels = pd.Series(km.labels_, index=category_counts.index, name='Cluster')
    neighborhoods = data[['Neighborhood', 'Latitude', 'Longitude']].groupby('Neighborhood').mean()
    labeled_neighborhoods = pd.concat([labels, neighborhoods], axis=1)
    return labeled_neighborhoods

labeled_neighborhoods = cluster_label(category_counts)
labeled_neighborhoods['Cluster'].value_counts()

0.0    104
4.0      6
1.0      3
2.0      2
3.0      2
Name: Cluster, dtype: int64

# Map neighborhoods, color-coded by cluster label
Clicking on a neighborhood marker will display a pop-up with the neighborhood name and the top 5 venue categories

In [15]:
def make_map(labeled_neighborhoods, top5):
    color_list = []
    for r, g, b in it.product(*[['00', 'cc']] * 3):
        color_list.append("#{}{}{}".format(r, g, b))
    color_list = color_list[1:-2]
    map_clusters = folium.Map(location=[labeled_neighborhoods['Latitude'].mean(), labeled_neighborhoods['Longitude'].mean()], zoom_start=11)
    for row in labeled_neighborhoods.iterrows():
        name, row = row
        cluster, lat, lng = row
        try:
            color = color_list[int(cluster)]
        except ValueError:
            color = '#000000'
        try:
            popup_text = "<b>{}</b><br>".format(name) + "<br>".join(["{}. {}".format(i, t) for i, t in zip(range(1, 6), top5.loc[name])])
        except KeyError:
            continue
        popup = folium.Popup(popup_text)
        marker = folium.CircleMarker([lat, lng], radius=6, popup=popup, color=color, fill=True, fillOpacity=0.7, min_width="100%", max_width='100%')
        marker.add_to(map_clusters)
    return map_clusters

In [16]:
map_clusters = make_map(labeled_neighborhoods, top5)
map_clusters

# Cluster neighborhoods based on venue category counts, but this time use broader categories

Foursquare has a category hierarchy. We will use this to translate specific venue categories (eg, Indian Restaurant) to a broader categories (eg, Food). This will decrease the dimensionality of the data used for k-means clustering

In [17]:
if not os.access('category_groups.json', os.R_OK):
    foursquare_categories = foursquare_template.format("categories")
    response = requests.get(foursquare_categories)
    response.raise_for_status()
    category_groups = response.json()
    with open('category_groups.json', 'w') as f:
        json.dump(category_groups, f)
else:
    with open('category_groups.json') as f:
        category_groups = json.load(f)

In [18]:
mapped_categories = {}
children = []
for parent in category_groups['response']['categories']:
    children.extend(parent['categories'])
    while children:
        child = children.pop()
        children.extend(child['categories'])
        mapped_categories[child['id']] = pd.Series({'Category ID': parent['id'], 'Category': parent['name']})

In [19]:
venues_alt = venues.copy()
venues_alt[['Category', 'Category ID']] = venues[['Category', 'Category ID']].apply(lambda x: mapped_categories[x.loc['Category ID']] 
                                                                                              if x.loc['Category ID'] in mapped_categories 
                                                                                              else x, 
                                                                                    axis=1)

Perform the same analysis as before:
1. Map 'Category' to boolean indicator features
2. Determine top 5 venue categories
3. Cluster neighborhoods based on venue frequencies
4. Map neighborhoods color-coded by cluster label

In [20]:
category_alt = venues_alt['Category']
category_alt = pd.get_dummies(category_alt, columns=['Category'], prefix="", prefix_sep="")
category_alt = pd.concat([venues_alt['Neighborhood'], category_alt], axis=1)
category_alt_grouped = category_alt.groupby('Neighborhood').mean()

In [21]:
top5_alt = get_top5(category_alt_grouped)
labeled_alt = cluster_label(category_alt_grouped)
labeled_alt['Cluster'].value_counts()

2.0    69
0.0    19
3.0    12
4.0     9
1.0     8
Name: Cluster, dtype: int64

In [22]:
map_alt = make_map(labeled_alt, top5_alt)
map_alt

# Cluster Summaries

In [23]:
def get_summary(labeled_neighborhoods, category_counts):
    cat_by_cluster = pd.concat([labeled_neighborhoods, category_counts], axis=1)
    cat_by_cluster = cat_by_cluster.drop(columns=['Latitude', 'Longitude']).groupby('Cluster').mean()
    summary = get_top5(cat_by_cluster)
    counts = labeled_neighborhoods['Cluster'].value_counts()
    counts.name = 'Neighborhood Count'
    summary = pd.concat([counts, summary], axis=1)
    summary.index.name = 'Cluster'
    return summary

## Using specific categories

In [24]:
get_summary(labeled_neighborhoods, category_counts).sort_values(by='Neighborhood Count', ascending=False)

,Neighborhood Count,Most Popular,2nd Most Popular,3rd Most Popular,4th Most Popular,5th Most Popular
Cluster,,,,,,
0.0,104,Coffee Shop,Park,Café,Bakery,Pizza Place
4.0,6,Coffee Shop,Restaurant,Park,Café,Clothing Store
1.0,3,Park,Lighting Store,ATM,Accessories Store,African Restaurant
2.0,2,Pharmacy,Grocery Store,ATM,Accessories Store,African Restaurant
3.0,2,Supermarket,ATM,Accessories Store,African Restaurant,Airport


## Using broad categories

In [25]:
get_summary(labeled_alt, category_alt_grouped).sort_values(by='Neighborhood Count', ascending=False)

,Neighborhood Count,Most Popular,2nd Most Popular,3rd Most Popular,4th Most Popular,5th Most Popular
Cluster,,,,,,
2.0,69,Food,Shop & Service,Outdoors & Recreation,Arts & Entertainment,Nightlife Spot
0.0,19,Food,Shop & Service,Outdoors & Recreation,Arts & Entertainment,Nightlife Spot
3.0,12,Shop & Service,Food,Outdoors & Recreation,Arts & Entertainment,College & University
4.0,9,Travel & Transport,Arts & Entertainment,Professional & Other Places,Nightlife Spot,Shop & Service
1.0,8,Outdoors & Recreation,Shop & Service,Food,Arts & Entertainment,College & University


# Discussion

It appears that using the broader category definitions helped improve clustering. Using the specific categories, the vast majority of the neighborhoods were assigned to a single cluster, meaning the distinction between neighborhoods was not as good. 

Since k-means clustering the venue category frequency assumes independence between venue categories (eg: frequency differences between 'Indian Restaurant' and 'Italian Restaurant' are treated with the same weight as frequency differences between 'Indian Restaurant' and 'Dog Park'), it makes sense that broadening the categories would work better. Otherwise, more specifically labeled data takes away from the true frequency count of similar venues. For example, restaurants with the style of food they serve in the name reduce the frequency of the simpler 'Restaurant' category and allow the frequency of these types of venues to get washed out when compared to categories for which no sub-sectioning is really possible, like 'Airport'. Additionally, using the broader categories allows us not to worry about whether a venue labeled 'Restaurant' because they serve a wide menu or because whoever tagged the venue as such did not bother to be more specific.